In [1]:
import csv
import os
import sys

import pandas as pd

utils_path = os.path.abspath(os.path.join('../..'))
if utils_path not in sys.path:
    sys.path.append(utils_path)
    
from utils.experiment import parse_barcodes, parse_exp_config
from utils.counts import normalize

In [2]:
bc_dict = parse_barcodes('../../ref/TruSeq_Stranded_RNA-Seq.csv', bc_id="AR")
exp_df = parse_exp_config('../../ref/2017-07-10_NextSeq.csv', bc_dict)
exp_df

,bc_id,bc_seq,bcm,sample,temp
0,AR2,CGATGT,False,utRho201,10
1,AR4,TGACCA,False,utRho202,10
2,AR5,ACAGTG,False,utRho203,10
3,AR6,GCCAAT,True,utRho204,10
4,AR7,CAGATC,True,utRho205,10
5,AR12,CTTGTA,True,utRho206,10
6,AR13,AGTCAA,False,utRho207,25
7,AR14,AGTTCC,False,utRho208,25
8,AR15,ATGTCA,False,utRho209,25
9,AR16,CCGTCC,True,utRho210,25


In [3]:
ct = pd.read_csv('../../results/utRho201/utRho201_sorted.counts',
                 header=None, names=['gene', 'counts'], sep='\t', skipfooter=5)
ct

/home/ilya/.venv/pydata3/lib/python3.5/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


,gene,counts
0,aaaD,11
1,aaaE,38
2,aaeA,150
3,aaeB,231
4,aaeR,606
5,aaeX,57
6,aas,2698
7,aat,491
8,abgA,70
9,abgB,113


In [4]:
def deseq_df(exp_df, cond, cond_names, res_dir='../results'):
    assert len(cond) == len(cond_names), 'cond and cond_names must be same length'
    df = pd.DataFrame()
    for cnd,name in zip(cond, cond_names):
        edf = exp_df[(exp_df.temp==cnd[0]) & (exp_df.bcm==cnd[1])]
        for i,row in enumerate(edf.iterrows()):
            i += 1
            _, sample = row
            filename = '{res_dir}/{sample}/{sample}_sorted.counts'.format(
                        res_dir=res_dir, sample=sample['sample'])
            ct = pd.read_csv(filename, header=None, names=['gene', 'counts'],
                            sep='\t', skipfooter=5, engine='python')
            if df.empty:
                df = ct.copy()
                df.rename(columns={'counts': '%s%i' % (name,i)}, inplace=True)
            else:
                df['%s%i' % (name,i)] = ct['counts']
    df.set_index('gene', inplace=True)
    df.index.names = [None]
    print(df)
    return df

In [5]:
d10bcm = deseq_df(exp_df, ((10, False),(10,True)), ('nobcm', 'bcm'), res_dir="../../results")
d10bcm.to_csv('../../results/2017-07-17/d10bcm.csv', index_label=False)

      nobcm1  nobcm2  nobcm3   bcm1   bcm2   bcm3
aaaD      11      15      10    208    206    182
aaaE      38      54      43   1996   1162    844
aaeA     150     253     144   5160   3001   2541
aaeB     231     421     228   9345   5553   4572
aaeR     606     892     687   6824   3627   4555
aaeX      57      99      55    971    513    567
aas     2698    4602    2875   6355   4054   5006
aat      491     708     591    897    734    946
abgA      70     127      77   2628   1776   2066
abgB     113     189     152   2314   1587   1815
abgR     372     657     372   2723   1731   2497
abgT      70     134     113   1891   1301   1596
abrB     444     723     426   2567   1366   2127
accA   11000   20394   10690  16778  10143  17477
accB   10774   17752   10209  10724   6421  11777
accC   23593   42065   23518  27651  16130  27536
accD   10645   19202   11524  11980   7658  11729
aceA    9785   15354   11406  10818   5031   8760
aceB   10978   18112   11957  13540   7058  11971


In [6]:
d44bcm = deseq_df(exp_df, ((44, False),(44, True)), ('nobcm', 'bcm'), res_dir="../../results")
d44bcm.to_csv('../../results/2017-07-17/d44bcm.csv', index_label=False)

      nobcm1  nobcm2  nobcm3    bcm1   bcm2    bcm3
aaaD      40      16      15     371    461     557
aaaE     194      81     110    7792   5202   10061
aaeA     704     247     450   10046   7776   13707
aaeB     756     305     501   18405  12605   23559
aaeR    1048     315     700    5722   5514    8706
aaeX     258      92     201    1294    976    1748
aas     6232    1897    3661    5249   5574    8122
aat     1125     333     686    1170   1012    1478
abgA     330     120     254    2086   1944    2983
abgB     434     174     260    2357   2227    3627
abgR     982     369     586    1976   1663    2924
abgT     170      58     125    2141   2033    3713
abrB     622     224     402    2698   2750    4253
accA   28522    8765   16365   15561  13236   20359
accB   27402    8196   15117    7379   7662    9527
accC   68814   21134   37357   23268  21818   28304
accD   41793   11821   23927   15892  13307   20153
aceA   25096    8773   15714    7405  10198   13670
aceB   33308

In [7]:
d37bcm = deseq_df(exp_df, ((37, False),(37,True)), ('nobcm', 'bcm'), res_dir="../../results")
d37bcm.to_csv('../../results/2017-07-17/d37bcm.csv', index_label=False)

      nobcm1  nobcm2  nobcm3   bcm1   bcm2   bcm3
aaaD      10       7      10    184    101     84
aaaE      11      11      16   4197   1837   1336
aaeA     106      85     110   4416   1939   1652
aaeB      90      85      85   8787   3960   3080
aaeR     147     184     198   4089   2511   1731
aaeX      36      40      55    623    342    219
aas     1279    1180    1577   7673   4696   3480
aat      287     286     428    668    435    382
abgA      39      60      59   1434    732    510
abgB      21      56      46   1239    732    569
abgR     214     326     318   1628   1192    886
abgT      37      27      12   1186    632    467
abrB     126     156     163   2043   1236    778
accA   10711   10702   12670  15363  12131   9683
accB   11911   12341   13740   9633   9461   7221
accC   29036   31568   35221  24325  23625  17133
accD   12828   13904   15934  12998  10444   8381
aceA    4303    5011    6321   6123   6070   3331
aceB    4540    5758    6198   8346   6991   4402


In [8]:
d37d10 = deseq_df(exp_df, ((37, False),(10,False)), ('norm', 'cs'), res_dir="../../results")
d37d10.to_csv('../../results/2017-07-17/d37d10.csv', index_label=False)

      norm1  norm2   norm3    cs1    cs2    cs3
aaaD     10      7      10     11     15     10
aaaE     11     11      16     38     54     43
aaeA    106     85     110    150    253    144
aaeB     90     85      85    231    421    228
aaeR    147    184     198    606    892    687
aaeX     36     40      55     57     99     55
aas    1279   1180    1577   2698   4602   2875
aat     287    286     428    491    708    591
abgA     39     60      59     70    127     77
abgB     21     56      46    113    189    152
abgR    214    326     318    372    657    372
abgT     37     27      12     70    134    113
abrB    126    156     163    444    723    426
accA  10711  10702   12670  11000  20394  10690
accB  11911  12341   13740  10774  17752  10209
accC  29036  31568   35221  23593  42065  23518
accD  12828  13904   15934  10645  19202  11524
aceA   4303   5011    6321   9785  15354  11406
aceB   4540   5758    6198  10978  18112  11957
aceE  78159  82499  104573  51411  83695

In [9]:
d37d44 = deseq_df(exp_df, ((37, False),(44,False)), ('norm', 'hs'), res_dir="../../results")
d37d44.to_csv('../../results/2017-07-17/d37d44.csv', index_label=False)

      norm1  norm2   norm3     hs1    hs2     hs3
aaaD     10      7      10      40     16      15
aaaE     11     11      16     194     81     110
aaeA    106     85     110     704    247     450
aaeB     90     85      85     756    305     501
aaeR    147    184     198    1048    315     700
aaeX     36     40      55     258     92     201
aas    1279   1180    1577    6232   1897    3661
aat     287    286     428    1125    333     686
abgA     39     60      59     330    120     254
abgB     21     56      46     434    174     260
abgR    214    326     318     982    369     586
abgT     37     27      12     170     58     125
abrB    126    156     163     622    224     402
accA  10711  10702   12670   28522   8765   16365
accB  11911  12341   13740   27402   8196   15117
accC  29036  31568   35221   68814  21134   37357
accD  12828  13904   15934   41793  11821   23927
aceA   4303   5011    6321   25096   8773   15714
aceB   4540   5758    6198   33308  12334   21670


In [7]:
def coldata(df, lib_type='single-end'):
    
    def set_cond(rec):
        return rec['cols'].strip('0123456789')
    
    cd = pd.DataFrame()
    cd['cols'] = df.columns.values
    cd['type'] = lib_type
    #conds = set([c.strip('0123456789') for c in df.columns.values])
    cd['condition'] = cd.apply(set_cond, axis=1)
    cd.set_index('cols', inplace=True)
    cd.index.names = [None]
    print(cd)
    return cd
    

In [10]:
cd = coldata(d10bcm, lib_type="paired-end")
cd.to_csv('../../results/2017-07-17/cd_10bcm.csv', index_label=False)

              type condition
nobcm1  paired-end     nobcm
nobcm2  paired-end     nobcm
nobcm3  paired-end     nobcm
bcm1    paired-end       bcm
bcm2    paired-end       bcm
bcm3    paired-end       bcm


In [11]:
cd = coldata(d44bcm, lib_type="paired-end")
cd.to_csv('../../results/2017-07-17/cd_44bcm.csv', index_label=False)

              type condition
nobcm1  paired-end     nobcm
nobcm2  paired-end     nobcm
nobcm3  paired-end     nobcm
bcm1    paired-end       bcm
bcm2    paired-end       bcm
bcm3    paired-end       bcm


In [11]:
cd = coldata(d37bcm)
cd.to_csv('../../results/2017-07-17/cd_37bcm.csv', index_label=False)

              type condition
nobcm1  single-end     nobcm
nobcm2  single-end     nobcm
nobcm3  single-end     nobcm
bcm1    single-end       bcm
bcm2    single-end       bcm
bcm3    single-end       bcm


In [12]:
cd = coldata(d37d10)
cd.to_csv('../../results/2017-07-17/cd_3710.csv', index_label=False)

             type condition
norm1  single-end      norm
norm2  single-end      norm
norm3  single-end      norm
cs1    single-end        cs
cs2    single-end        cs
cs3    single-end        cs


In [13]:
cd = coldata(d37d44)
cd.to_csv('../../results/2017-07-17/cd_3744.csv', index_label=False)

             type condition
norm1  single-end      norm
norm2  single-end      norm
norm3  single-end      norm
hs1    single-end        hs
hs2    single-end        hs
hs3    single-end        hs
